In [10]:
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Average, Dropout
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.datasets import cifar10

import numpy as np

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, num_classes=10)

In [12]:
print(
    'x_train shape: {} | y_train shape: {}\nx_test shape : {} | y_test shape : {}'.format(x_train.shape, y_train.shape,
                                                                                          x_test.shape, y_test.shape))

x_train shape: (50000, 32, 32, 3) | y_train shape: (50000, 10)
x_test shape : (10000, 32, 32, 3) | y_test shape : (10000, 1)


In [13]:
input_shape = x_train[0, :, :, :].shape
model_input = Input(shape=input_shape)

In [14]:
def conv_pool_cnn(model_input):
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding='same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)

    model = Model(model_input, x, name='conv_pool_cnn')
    model.summary()
    return model

In [17]:
conv_pool_cnn_model = conv_pool_cnn(model_input)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 96)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 15, 15, 192)       166080    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 15, 15, 192)       331968    
__________

In [24]:
def compile_and_train(model, num_epochs):
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc'])
    filepath = '../models/assemble_learning/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True,
                                 mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='/home/Bureaux/Project/Keras-DL-server/logs/assemble_learning/', histogram_freq=0, batch_size=128)
    history = model.fit(x=x_train, y=y_train, batch_size=128, epochs=num_epochs, verbose=1,
                        callbacks=[checkpoint, tensor_board], validation_split=0.2)
    return history

In [25]:
_ = compile_and_train(conv_pool_cnn_model, num_epochs=20)


Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 7s 184us/step - loss: 1.4576 - acc: 0.4572 - val_loss: 1.2793 - val_acc: 0.5269

In [28]:
def evaluate_error(model):
    pred = model.predict(x_test, batch_size=128)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1)  # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]

    return error

In [29]:
evaluate_error(conv_pool_cnn_model)

0.2243

In [30]:
def all_cnn(model_input):
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding='same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding='same', strides=2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding='same', strides=2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)

    model = Model(model_input, x, name='all_cnn')
    model.summary()
    return model

In [31]:
all_cnn_model = all_cnn(model_input)
_ = compile_and_train(all_cnn_model, num_epochs=20)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 16, 16, 192)       166080    
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 192)       331968    
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 8, 8, 192)         331968    
__________

In [32]:
evaluate_error(all_cnn_model)

0.2166

In [33]:
def nin_cnn(model_input):
    #mlpconv block 1
    x = Conv2D(32, (5, 5), activation='relu', padding='valid')(model_input)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)

    #mlpconv block2
    x = Conv2D(64, (3, 3), activation='relu', padding='valid')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)

    #mlpconv block3
    x = Conv2D(128, (3, 3), activation='relu', padding='valid')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)

    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)

    model = Model(model_input, x, name='nin_cnn')
    model.summary()
    return model


nin_cnn_model = nin_cnn(model_input)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [34]:
_ = compile_and_train(nin_cnn_model, num_epochs=20)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 4s 104us/step - loss: 1.9243 - acc: 0.2650 - val_loss: 1.7165 - val_acc: 0.3404

In [35]:
evaluate_error(nin_cnn_model)

0.3416

In [36]:
conv_pool_cnn_model = conv_pool_cnn(model_input)
all_cnn_model = all_cnn(model_input)
nin_cnn_model = nin_cnn(model_input)

conv_pool_cnn_model.load_weights('/home/Bureaux/Project/Keras-DL-server/models/conv_pool_cnn.19-0.09.hdf5')
all_cnn_model.load_weights('/home/Bureaux/Project/Keras-DL-server/models/all_cnn.20-0.08.hdf5')
nin_cnn_model.load_weights('/home/Bureaux/Project/Keras-DL-server/models/nin_cnn.20-1.03.hdf5')

models = [conv_pool_cnn_model, all_cnn_model, nin_cnn_model]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 96)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 15, 15, 192)       166080    
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 15, 15, 192)       331968    
__________

In [37]:
def ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)

    model = Model(model_input, y, name='ensemble')

    return model

In [38]:
ensemble_model = ensemble(models, model_input)

In [39]:
evaluate_error(ensemble_model)

0.176

In [40]:
pair_A = [conv_pool_cnn_model, all_cnn_model]
pair_B = [conv_pool_cnn_model, nin_cnn_model]
pair_C = [all_cnn_model, nin_cnn_model]
pair_A_ensemble_model = ensemble(pair_A, model_input)
evaluate_error(pair_A_ensemble_model)

0.172

In [41]:
pair_B_ensemble_model = ensemble(pair_B, model_input)
evaluate_error(pair_B_ensemble_model)

0.1875

In [42]:
pair_C_ensemble_model = ensemble(pair_C, model_input)
evaluate_error(pair_C_ensemble_model)

0.2089